
## **stack and unstack** in pandas

This note is based on [python for data analysis](http://localhost:8888/notebooks/ch07.ipynb)

In the book, it says

> stack change column to row

> and unstack chang row to column

It is hard to understand


更容易理解的方式是：

- stack and unstack 用來操作multiindex, change Series into
Dataframe or change one type DataFrame to another type
- 默認用最內層的index，且將其放在最外層

> 最內層---最右的index

> 最外層

> For Series, it is the column

> For DataFrame, it is the rightmost index

- In practice, check the examples and make sure it is right


補充：

- 類似 melt and cast in a package of R， melt 成單列形式，然後就可以任意操作了

In [1]:
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [2]:
data.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

- **wrong, for data only has one single index: 'state' **

*IndexError: Too many levels: Index has only 1 level, not 2*

- 在构造多index的DataFrame中，可以更清楚的理解index的level 0, level 1

For example, 
'index=[['a','a','b','b'],['at','ay','bt','by']]'
中 
'['a','a','b','b']'
是level 0, i.e., index[0]

In [3]:
df = DataFrame(np.arange(4),index=[['a','a','b','b'],['at','ay','bt','by']])
df

0
a at  0
  ay  1
b bt  2
  by  3

In [4]:
df.index

MultiIndex(levels=[['a', 'b'], ['at', 'ay', 'bt', 'by']],
           labels=[[0, 0, 1, 1], [0, 1, 2, 3]])

data.stack(1) is wrong

** newdata = data.stack() has two level index.**
- so it can unstack(0) and unstack(1) 

In [6]:
data.stack().unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [7]:
data.stack().unstack(1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [8]:
result = data.stack()
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [14]:
h = df.unstack()
h['city'] = ['sh','hz']
h# h.set_index(['state','city'])

side     left           right           city
number    one two three   one two three     
state                                       
Ohio        0   1     2     5   6     7   sh
Colorado    3   4     5     8   9    10   hz

In [35]:
df.stack()

state     number  side 
Ohio      one     left      0
                  right     5
          two     left      1
                  right     6
          three   left      2
                  right     7
Colorado  one     left      3
                  right     8
          two     left      4
                  right     9
          three   left      5
                  right    10
dtype: int64

In [36]:
df.stack().unstack(0) # df.stack().unstack('state')  

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [37]:
df.stack().unstack(0) # df.stack().unstack('number') 

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [38]:
df.stack().unstack(0) # df.stack().unstack('side') 

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [39]:
df.stack().unstack('side') # check, same as above

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

** something about the multiindex and multi-clomuns DataFrame**

- stack  level[-1] of column --> level[-1] of index  上到左（最终是排成一列）

- unstack    level[-1] of index --> level[-1] of column  左到上（最终是排成一行）

-  最终的结果都是 Series

In [32]:
result = data.stack()
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))
h = df.unstack()
h

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [33]:
h['city'] = ['sh','sc']
h=h.set_index('city',append=True)
h

side          left           right          
number         one two three   one two three
state    city                               
Ohio     sh      0   1     2     5   6     7
Colorado sc      3   4     5     8   9    10

In [34]:
h.stack()

side                  left  right
state    city number             
Ohio     sh   one        0      5
              two        1      6
              three      2      7
Colorado sc   one        3      8
              two        4      9
              three      5     10

In [35]:
h.stack().stack()

state     city  number  side 
Ohio      sh    one     left      0
                        right     5
                two     left      1
                        right     6
                three   left      2
                        right     7
Colorado  sc    one     left      3
                        right     8
                two     left      4
                        right     9
                three   left      5
                        right    10
dtype: int64

In [36]:
h.unstack()

side     left                           right                          
number    one       two      three        one       two      three     
city       sc   sh   sc   sh    sc   sh    sc   sh   sc   sh    sc   sh
state                                                                  
Colorado  3.0  NaN  4.0  NaN   5.0  NaN   8.0  NaN  9.0  NaN  10.0  NaN
Ohio      NaN  0.0  NaN  1.0   NaN  2.0   NaN  5.0  NaN  6.0   NaN  7.0

In [41]:
huu = h.unstack().unstack()
huu

side   number  city  state   
left   one     sc    Colorado     3.0
                     Ohio         NaN
               sh    Colorado     NaN
                     Ohio         0.0
       two     sc    Colorado     4.0
                     Ohio         NaN
               sh    Colorado     NaN
                     Ohio         1.0
       three   sc    Colorado     5.0
                     Ohio         NaN
               sh    Colorado     NaN
                     Ohio         2.0
right  one     sc    Colorado     8.0
                     Ohio         NaN
               sh    Colorado     NaN
                     Ohio         5.0
       two     sc    Colorado     9.0
                     Ohio         NaN
               sh    Colorado     NaN
                     Ohio         6.0
       three   sc    Colorado    10.0
                     Ohio         NaN
               sh    Colorado     NaN
                     Ohio         7.0
dtype: float64

In [42]:
huu?